In [85]:
#Group: Dan Bezabhe, Yeabsira Sahlu
import sqlite3

connection = sqlite3.connect('superhero_db.sqlite')
cursor = connection.cursor()

print("Database connected successfully!")


Database connected successfully!


In [86]:
cursor.execute("DROP TABLE IF EXISTS battles;")
cursor.execute("DROP TABLE IF EXISTS heroes;")
cursor.execute("DROP TABLE IF EXISTS villains;")

cursor.execute('''
    CREATE TABLE heroes (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        power TEXT,
        team TEXT
    );
''')

cursor.execute('''
    CREATE TABLE villains (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        evil_plan TEXT
    );
''')

cursor.execute('''
    CREATE TABLE battles (
        id INTEGER PRIMARY KEY,
        hero_id INTEGER NOT NULL,
        villain_id INTEGER NOT NULL,
        outcome TEXT NOT NULL,
        FOREIGN KEY (hero_id) REFERENCES heroes(id),
        FOREIGN KEY (villain_id) REFERENCES villains(id)
    );
''')

print("tables created successfully")


tables created successfully


In [87]:
import csv

heroes_data = []
villains_data = []
battles_data = []

with open('heroes.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        heroes_data.append((
            int(row['id']),
            row['name'],
            row['power'],
            row['team']
        ))

with open('villains.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        villains_data.append((
            int(row['id']),
            row['name'],
            row['evil_plan']
        ))

with open('battles.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        battles_data.append((
            int(row['id']),
            int(row['hero_id']),
            int(row['villain_id']),
            row['outcome']
        ))

print("CSV files loaded into Python lists")


CSV files loaded into Python lists


In [88]:
cursor.executemany('''
    INSERT INTO heroes (id, name, power, team)
    VALUES (?, ?, ?, ?);
''', heroes_data)

cursor.executemany('''
    INSERT INTO villains (id, name, evil_plan)
    VALUES (?, ?, ?);
''', villains_data)

cursor.executemany('''
    INSERT INTO battles (id, hero_id, villain_id, outcome)
    VALUES (?, ?, ?, ?);
''', battles_data)

connection.commit()

print("data inserted successfully into heroes, villains, and battles")


data inserted successfully into heroes, villains, and battles


In [89]:
print("Retrieve all superheroes and their powers.")
cursor.execute("SELECT name, power FROM heroes;")
rows = cursor.fetchall()
for (hero_name, power) in rows:
    print(f"Hero: {hero_name}, Power: {power}")



Retrieve all superheroes and their powers.
Hero: Spider-Man, Power: Web-slinging
Hero: Iron Man, Power: Powered Armor
Hero: Batman, Power: Genius Detective
Hero: Superman, Power: Super Strength
Hero: Thor, Power: God of Thunder
Hero: Wonder Woman, Power: Amazonian Strength
Hero: Black Panther, Power: Enhanced Agility
Hero: The Flash, Power: Super Speed
Hero: Hulk, Power: Super Strength
Hero: Doctor Strange, Power: Mystic Arts


In [90]:
print("Find all battles where the hero won.")
cursor.execute('''
    SELECT b.id, h.name AS hero_name, v.name AS villain_name, b.outcome
    FROM battles b
    JOIN heroes h ON b.hero_id = h.id
    JOIN villains v ON b.villain_id = v.id
    WHERE b.outcome = "Hero Wins";
''')
rows = cursor.fetchall()
for battle_id, hero_name, villain_name, outcome in rows:
    print(f"Battle {battle_id}: {hero_name} vs {villain_name}, Outcome: {outcome}")


Find all battles where the hero won.
Battle 2: Hulk vs Joker, Outcome: Hero Wins
Battle 5: Spider-Man vs Darkseid, Outcome: Hero Wins
Battle 7: Hulk vs Venom, Outcome: Hero Wins
Battle 8: Batman vs Thanos, Outcome: Hero Wins
Battle 9: Black Panther vs Loki, Outcome: Hero Wins
Battle 13: The Flash vs Magneto, Outcome: Hero Wins
Battle 20: Batman vs Lex Luthor, Outcome: Hero Wins
Battle 29: Batman vs Loki, Outcome: Hero Wins
Battle 30: Thor vs Loki, Outcome: Hero Wins


In [91]:
print("List all villains and their evil plans.")
cursor.execute("SELECT name, evil_plan FROM villains;")
rows = cursor.fetchall()
for villain_name, plan in rows:
    print(f"Villain: {villain_name}, Evil Plan: {plan}")


List all villains and their evil plans.
Villain: Green Goblin, Evil Plan: Terrorize New York
Villain: Thanos, Evil Plan: Collect all Infinity Stones
Villain: Joker, Evil Plan: Spread chaos in Gotham
Villain: Lex Luthor, Evil Plan: Defeat Superman
Villain: Loki, Evil Plan: Trick humanity into worshiping him
Villain: Ultron, Evil Plan: AI world domination
Villain: Darkseid, Evil Plan: Control the universe
Villain: Venom, Evil Plan: Consume Spider-Man
Villain: Magneto, Evil Plan: Mutant supremacy
Villain: Red Skull, Evil Plan: Revive Hydra


In [92]:
print("Determine which superhero has fought the most battles.")
cursor.execute('''
    SELECT h.name, COUNT(*) AS total_battles
    FROM battles b
    JOIN heroes h ON b.hero_id = h.id
    GROUP BY h.id
    ORDER BY total_battles DESC
    LIMIT 1;
''')
row = cursor.fetchone()
if row:
    hero_name, total_battles = row
    print(f"{hero_name} has fought the most battles ({total_battles}).")
else:
    print("No battles found.")


Determine which superhero has fought the most battles.
Batman has fought the most battles (7).


In [93]:
print("Battle locations table: ")


cursor.execute("DROP TABLE IF EXISTS battle_locations;")

cursor.execute('''
    CREATE TABLE battle_locations (
        location_id INTEGER PRIMARY KEY AUTOINCREMENT,
        battle_id INTEGER NOT NULL,
        location_name TEXT NOT NULL,
        FOREIGN KEY(battle_id) REFERENCES battles(id)
    );
''')

battle_locations_data = [
    (1, "New York City"),
    (2, "Gotham City"),
    (3, "Metropolis"),

]

cursor.executemany('''
    INSERT INTO battle_locations (battle_id, location_name)
    VALUES (?, ?);
''', battle_locations_data)

connection.commit()

print("Sample data inserted into battle_locations.")


print("battle_locations table created successfully!")



Battle locations table: 
Sample data inserted into battle_locations.
battle_locations table created successfully!


In [94]:
print("Battles Locations: ")

cursor.execute('''
    SELECT b.id AS battle_id,
           bl.location_name,
           h.name AS hero,
           v.name AS villain,
           b.outcome
    FROM battles b
    JOIN battle_locations bl ON b.id = bl.battle_id
    JOIN heroes h ON b.hero_id = h.id
    JOIN villains v ON b.villain_id = v.id
    ORDER BY b.id ASC;
''')

rows = cursor.fetchall()
for battle_id, location_name, hero_name, villain_name, outcome in rows:
    print(f"Battle {battle_id} @ {location_name}: {hero_name} vs. {villain_name} => {outcome}")



Battles Locations: 
Battle 1 @ New York City: Spider-Man vs. Red Skull => Villain Wins
Battle 2 @ Gotham City: Hulk vs. Joker => Hero Wins
Battle 3 @ Metropolis: The Flash vs. Green Goblin => Draw


In [95]:
connection.close()